# To Predict Depression Among Patients Admitted to ICU 

## Part I: Data Preprocessing

### Import packages

In [1]:
# Adjust notebook settings to widen the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>"))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### Import modules/datasets

In [7]:
# every unique hospitalization for each patient in the database (defines HADM_ID_
admissions = pd.read_csv('data/ADMISSIONS.csv')
# Diagnosis Related Groups (DRG), which are used by the hospital for billing purposes.
drgcodes = pd.read_csv("data/DRGCODES.csv")
# Deidentified notes, including nursing and physician notes, ECG reports, imaging reports, and discharge summaries.
noteevents = pd.read_csv("data/NOTEEVENTS.csv")
# every unique patient in the database (defines subject_id)
patients = pd.read_csv("data/PATIENTS.csv")
# the clinical service under which a patient is registered
services = pd.read_csv("data/SERVICES.csv")
# Medications ordered, and not necessarily administered, for a given patient
prescriptions = pd.read_csv("data/PRESCRIPTIONS.csv")
# Ground truth dataset
phenotypes = pd.read_csv("data/GROUND_TRUTH.csv")

In [8]:
# lowercase all strings inside of a dataframe to lowercase
admissions = admissions.apply(lambda x: x.astype(str).str.lower())
drgcodes = drgcodes.apply(lambda x: x.astype(str).str.lower())
noteevents = noteevents.apply(lambda x: x.astype(str).str.lower())
patients = patients.apply(lambda x: x.astype(str).str.lower())
services = services.apply(lambda x: x.astype(str).str.lower())
prescriptions = prescriptions.apply(lambda x: x.astype(str).str.lower())
phenotypes = phenotypes.apply(lambda x: x.astype(str).str.lower())

# lowercase columns in all dataframes
admissions.columns = admissions.columns.str.lower()
drgcodes.columns = drgcodes.columns.str.lower()
noteevents.columns = noteevents.columns.str.lower()
patients.columns = patients.columns.str.lower()
services.columns = services.columns.str.lower()
prescriptions.columns = prescriptions.columns.str.lower()
phenotypes.columns = phenotypes.columns.str.lower()

In [9]:
# decrease the datasets by subsetting the records which ID is in phenotypes dataset
admissions_reduced = admissions[admissions['subject_id'].isin(phenotypes['subject_id'])]
drgcodes_reduced = drgcodes[drgcodes['subject_id'].isin(phenotypes['subject_id'])]
noteevents_reduced = noteevents[noteevents['subject_id'].isin(phenotypes['subject_id'])]
patients_reduced = patients[patients['subject_id'].isin(phenotypes['subject_id'])]
services_reduced = services[services['subject_id'].isin(phenotypes['subject_id'])]
prescriptions_reduced = prescriptions[prescriptions['subject_id'].isin(phenotypes['subject_id'])]

admissions_reduced = admissions_reduced.reset_index(drop=True)
drgcodes_reduced = drgcodes_reduced.reset_index(drop=True)
noteevents_reduced = noteevents_reduced.reset_index(drop=True)
patients_reduced = patients_reduced.reset_index(drop=True)
services_reduced = services_reduced.reset_index(drop=True)
prescriptions_reduced = prescriptions_reduced.reset_index(drop=True)

### Functions

In [6]:
# function to get unique values
def unique(list1):
    x = np.array(list1)
    print(np.unique(x))

### Clean dataset: phenotypes

In [22]:
# Only keep the interested outcome feature
phenotypes_reduced = phenotypes[['hadm_id','subject_id','depression']]
# Drop duplicated records by subject_id and hadm_id
phenotypes_reduced = phenotypes_reduced.drop_duplicates(subset=['subject_id','hadm_id'], ignore_index = True)

In [23]:
# Size of phenotypes_reduced
phenotypes_reduced.shape

(813, 3)

In [24]:
phenotypes_reduced.head()

,hadm_id,subject_id,depression
0,100103,3365,0
1,100137,27290,0
2,100473,5525,0
3,100485,41515,1
4,100548,2265,0


### Clean dataset: admissions

#### Regarding diagnosis feature from admission: 

15,693 distinct diagnoses for 58,976 admissions. The diagnoses can be very informative (e.g. chronic kidney failure) or quite vague (e.g. weakness). Final diagnoses for a patient’s hospital stay are coded on discharge and can be found in the DIAGNOSES_ICD table. While this field can provide information about the status of a patient on hospital admission, it is not recommended to use it to stratify patients.


In [25]:
# Size of admissions_reduced
admissions_reduced.shape

(1944, 19)

In [26]:
# Since the dates and times from the database are deidentified -- create new features to get the time difference
admissions_reduced['edouttime'] = pd.to_datetime(admissions_reduced['edouttime'])
admissions_reduced['edregtime'] = pd.to_datetime(admissions_reduced['edregtime'])
admissions_reduced['length_ed'] = (admissions_reduced['edouttime'] - admissions_reduced['edregtime']).dt.days
admissions_reduced['dischtime'] = pd.to_datetime(admissions_reduced['dischtime'])
admissions_reduced['admittime'] = pd.to_datetime(admissions_reduced['admittime'])
admissions_reduced['length_admit'] = (admissions_reduced['dischtime'] - admissions_reduced['admittime']).dt.days
# Drop time-related features used to create new features
admissions_reduced = admissions_reduced.drop(['edregtime', 'edouttime', 'dischtime', 'admittime'], axis = 1)

In [27]:
# Create aggregate, dummy, and new variables for admission df to create one row per id
just_dummies = pd.get_dummies(admissions_reduced['admission_type'], prefix='admission_type')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for admission location
just_dummies = pd.get_dummies(admissions_reduced['admission_location'], prefix='admission_loc')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for discharge location
just_dummies = pd.get_dummies(admissions_reduced['discharge_location'], prefix='discharge_loc')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for insurance
just_dummies = pd.get_dummies(admissions_reduced['insurance'], prefix='insurance')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for religions
just_dummies = pd.get_dummies(admissions_reduced['religion'], prefix='religion')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for language
just_dummies = pd.get_dummies(admissions_reduced['language'], prefix='language')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for marital_status
just_dummies = pd.get_dummies(admissions_reduced['marital_status'], prefix='marital_status')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)
# Create dummy variables for ethnicity
just_dummies = pd.get_dummies(admissions_reduced['ethnicity'], prefix='ethnicity')
admissions_reduced = pd.concat([admissions_reduced, just_dummies], axis=1)

In [28]:
# Remove features used to dummy variables
admissions_reduced = admissions_reduced.drop(['row_id', 'deathtime', 'diagnosis', 'religion', 'language','marital_status', 'ethnicity', 'insurance', 'admission_location', 'discharge_location', 'admission_type'], axis = 1)

In [29]:
admissions_reduced.head(2)

,subject_id,hadm_id,hospital_expire_flag,has_chartevents_data,length_ed,length_admit,admission_type_elective,admission_type_emergency,admission_type_urgent,admission_loc_clinic referral/premature,admission_loc_emergency room admit,admission_loc_phys referral/normal deli,admission_loc_transfer from hosp/extram,admission_loc_transfer from other healt,admission_loc_transfer from skilled nur,discharge_loc_dead/expired,discharge_loc_disc-tran cancer/chldrn h,discharge_loc_disch-tran to psych hosp,discharge_loc_home,discharge_loc_home health care,discharge_loc_home with home iv providr,discharge_loc_hospice-home,discharge_loc_hospice-medical facility,discharge_loc_icf,discharge_loc_left against medical advi,discharge_loc_long term care hospital,discharge_loc_other facility,discharge_loc_rehab/distinct part hosp,discharge_loc_short term hospital,discharge_loc_snf,insurance_government,insurance_medicaid,insurance_medicare,insurance_private,insurance_self pay,religion_baptist,religion_buddhist,religion_catholic,religion_christian scientist,religion_episcopalian,religion_greek orthodox,religion_hebrew,religion_jehovah's witness,religion_jewish,religion_muslim,religion_nan,religion_not specified,religion_other,religion_protestant quaker,religion_romanian east. orth,religion_unitarian-universalist,religion_unobtainable,language_*chi,language_*hun,language_*man,language_arab,language_camb,language_cant,language_cape,language_engl,language_fren,language_gree,language_hait,language_ital,language_nan,language_pers,language_port,language_ptun,language_russ,language_span,language_urdu,marital_status_divorced,marital_status_life partner,marital_status_married,marital_status_nan,marital_status_separated,marital_status_single,marital_status_unknown (default),marital_status_widowed,ethnicity_asian,ethnicity_asian - asian indian,ethnicity_asian - chinese,ethnicity_black/african,ethnicity_black/african american,ethnicity_black/cape verdean,ethnicity_black/haitian,ethnicity_hispanic or latino,ethnicity_hispanic/latino - guatemalan,ethnicity_hispanic/latino - puerto rican,ethnicity_other,ethnicity_patient declined to answer,ethnicity_portuguese,ethnicity_unable to obtain,ethnicity_unknown/not specified,ethnicity_white,ethnicity_white - brazilian,ethnicity_white - eastern european,ethnicity_white - russian
0,368,105889,0,1,0.0,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,368,138061,0,1,1.0,5,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [30]:
admissions_reduced["hospital_expire_flag"] = admissions_reduced.hospital_expire_flag.astype(float)
admissions_reduced["has_chartevents_data"] = admissions_reduced.has_chartevents_data.astype(float)

In [31]:
# Size of cleaned admissions_reduced dataset
admissions_reduced.shape

(1944, 98)

### Clean dataset: patients

In [32]:
# Size of patients_reduced dataset
patients_reduced.shape

(473, 8)

In [33]:
# Create numerical code for string variables in the gender feature
patients_reduced.gender[patients_reduced.gender == 'm'] = 1
patients_reduced.gender[patients_reduced.gender == 'f'] = 0

In [34]:
# Removing PIH features that had been deidentified
patients_reduced = patients_reduced.drop(['row_id', 'dob', 'dod', 'dod_hosp', 'dod_ssn'], axis = 1)

In [35]:
# Final features left for patients_reduced dataset
patients_reduced.head(2)

,subject_id,gender,expire_flag
0,690,1,1
1,704,1,1


### Cleaning dataset:  drgcodes

In [36]:
drgcodes_reduced.shape

(1941, 4)

In [37]:
drgcodes_reduced.head(3)

,subject_id,hadm_id,avg_drg_mortality,avg_drg_severity
0,6451,183196,0.0,0.0
1,4655,143283,0.0,0.0
2,23000,132906,0.0,0.0


In [ ]:
# # Create dummy variables for drg_code
# just_dummies = pd.get_dummies(drgcodes_reduced['drg_code'], prefix='drg_code')
# drgcodes_reduced = pd.concat([drgcodes_reduced, just_dummies], axis=1)

# # Create dummy variables for drg_code
# just_dummies = pd.get_dummies(drgcodes_reduced['drg_type'], prefix='drg_type')
# drgcodes_reduced = pd.concat([drgcodes_reduced, just_dummies], axis=1)

In [11]:
# # Transform object to numerical features
# drgcodes_reduced['drg_mortality'] = pd.to_numeric(drgcodes_reduced.drg_mortality, errors='coerce').fillna(0, downcast='infer').astype('Int32')
# drgcodes_reduced['drg_severity'] = pd.to_numeric(drgcodes_reduced.drg_severity, errors='coerce').fillna(0, downcast='infer').astype('Int32')

In [12]:
# # In order to have one record for each unique combination of subject_id and hadm_id, mean of the drg_mortality and drg_severity
# # are calculated
# drgcodes_reduced['avg_drg_mortality'] = drgcodes_reduced.groupby(['subject_id', 'hadm_id']).drg_mortality.transform('mean')
# drgcodes_reduced['avg_drg_severity'] = drgcodes_reduced.groupby(['subject_id', 'hadm_id']).drg_severity.transform('mean')

In [13]:
# drgcodes_reduced['avg_drg_mortality'] = drgcodes_reduced.avg_drg_mortality.astype(float)
# drgcodes_reduced['avg_drg_severity'] = drgcodes_reduced.avg_drg_severity.astype(float)

In [41]:
# Drop duplicates by comparing subject_id and hadm_id
drgcodes_reduced = drgcodes_reduced.drop_duplicates(subset=['subject_id','hadm_id'], ignore_index = True)

In [15]:
drgcodes_reduced = drgcodes_reduced.drop(['row_id', 'description', 'drg_code', 'drg_type', 'drg_severity', 'drg_mortality' ], axis = 1)

In [42]:
# Final size of the drgcodes_reduced
drgcodes_reduced.shape

(1941, 4)

### Merge Datasets

In [43]:
phenotypes_reduced.shape

(813, 3)

In [44]:
main = pd.merge(admissions_reduced, phenotypes_reduced,
                how ='right',
                on = ['subject_id', 'hadm_id'])

main = pd.merge(main, patients_reduced,
                how ='left',
                on = ['subject_id'])

main = pd.merge(main, drgcodes_reduced,
                how ='left',
                on = ['subject_id', 'hadm_id'])

In [45]:
main["expire_flag"] = main.expire_flag.astype(float)
main['gender'] = main.gender.astype(int)

In [46]:
main.shape # final dataset (813, 742)

(813, 103)

### Missingness of the final merged datasets

In [47]:
main.isnull().mean() # length_ed had ~30% missingness

subject_id                                  0.000000
hadm_id                                     0.000000
hospital_expire_flag                        0.000000
has_chartevents_data                        0.000000
length_ed                                   0.303813
length_admit                                0.000000
admission_type_elective                     0.000000
admission_type_emergency                    0.000000
admission_type_urgent                       0.000000
admission_loc_clinic referral/premature     0.000000
admission_loc_emergency room admit          0.000000
admission_loc_phys referral/normal deli     0.000000
admission_loc_transfer from hosp/extram     0.000000
admission_loc_transfer from other healt     0.000000
admission_loc_transfer from skilled nur     0.000000
discharge_loc_dead/expired                  0.000000
discharge_loc_disc-tran cancer/chldrn h     0.000000
discharge_loc_disch-tran to psych hosp      0.000000
discharge_loc_home                          0.

In [48]:
main.head(5)

,subject_id,hadm_id,hospital_expire_flag,has_chartevents_data,length_ed,length_admit,admission_type_elective,admission_type_emergency,admission_type_urgent,admission_loc_clinic referral/premature,admission_loc_emergency room admit,admission_loc_phys referral/normal deli,admission_loc_transfer from hosp/extram,admission_loc_transfer from other healt,admission_loc_transfer from skilled nur,discharge_loc_dead/expired,discharge_loc_disc-tran cancer/chldrn h,discharge_loc_disch-tran to psych hosp,discharge_loc_home,discharge_loc_home health care,discharge_loc_home with home iv providr,discharge_loc_hospice-home,discharge_loc_hospice-medical facility,discharge_loc_icf,discharge_loc_left against medical advi,discharge_loc_long term care hospital,discharge_loc_other facility,discharge_loc_rehab/distinct part hosp,discharge_loc_short term hospital,discharge_loc_snf,insurance_government,insurance_medicaid,insurance_medicare,insurance_private,insurance_self pay,religion_baptist,religion_buddhist,religion_catholic,religion_christian scientist,religion_episcopalian,religion_greek orthodox,religion_hebrew,religion_jehovah's witness,religion_jewish,religion_muslim,religion_nan,religion_not specified,religion_other,religion_protestant quaker,religion_romanian east. orth,religion_unitarian-universalist,religion_unobtainable,language_*chi,language_*hun,language_*man,language_arab,language_camb,language_cant,language_cape,language_engl,language_fren,language_gree,language_hait,language_ital,language_nan,language_pers,language_port,language_ptun,language_russ,language_span,language_urdu,marital_status_divorced,marital_status_life partner,marital_status_married,marital_status_nan,marital_status_separated,marital_status_single,marital_status_unknown (default),marital_status_widowed,ethnicity_asian,ethnicity_asian - asian indian,ethnicity_asian - chinese,ethnicity_black/african,ethnicity_black/african american,ethnicity_black/cape verdean,ethnicity_black/haitian,ethnicity_hispanic or latino,ethnicity_hispanic/latino - guatemalan,ethnicity_hispanic/latino - puerto rican,ethnicity_other,ethnicity_patient declined to answer,ethnicity_portuguese,ethnicity_unable to obtain,ethnicity_unknown/not specified,ethnicity_white,ethnicity_white - brazilian,ethnicity_white - eastern european,ethnicity_white - russian,depression,gender,expire_flag,avg_drg_mortality,avg_drg_severity
0,3365,100103,0.0,1.0,NaN,10,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.000000,0.000000
1,27290,100137,0.0,1.0,NaN,7,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.0,2.000000,2.000000
2,5525,100473,0.0,1.0,0.0,16,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.0,2.666667,2.666667
3,41515,100485,0.0,1.0,NaN,4,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.0,0.666667,1.333333
4,2265,100548,0.0,1.0,0.0,13,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1.0,0.000000,0.000000


In [49]:
main['length_ed'] = main['length_ed'].fillna(0) # not entirely sure about this 

### Restructuring -- Move the outcome variable to be the last column in the dataset

In [50]:
cols = list(main.columns.values)
cols.pop(cols.index('depression'))
main = main[cols+['depression']]

In [51]:
main["hospital_expire_flag"] = main.hospital_expire_flag.astype(float)
main["has_chartevents_data"] = main.has_chartevents_data.astype(float)
main["expire_flag"] = main.expire_flag.astype(float)
main['gender'] = main.gender.astype(int)
main['avg_drg_mortality'] = main.avg_drg_mortality.astype(float)
main['avg_drg_severity'] = main.avg_drg_severity.astype(float)
main['depression'] = main.depression.astype(int)

#### Set X as features and y as the outcome

In [52]:
main = main.drop(['subject_id', 'hadm_id'], axis = 1)

# High correlation filter

This dimensionality reduction algorithm tries to discard inputs that are very similar to others. In simple words, if your opinion is same as your boss, one of you is not required. If the value of two input parameters is always the same, it means they represent the same entity. Then we do not need two parameters there. Just one should be enough.

In technical words, if there is a very high correlation between two input variables, we can safely drop one of them.

In [53]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(main, 10))

Top Absolute Correlations
language_hait             ethnicity_black/haitian                    1.000000
hospital_expire_flag      discharge_loc_dead/expired                 1.000000
avg_drg_mortality         avg_drg_severity                           0.955766
religion_buddhist         language_camb                              0.865491
admission_type_elective   admission_type_emergency                   0.851231
language_engl             language_nan                               0.824003
language_port             ethnicity_white - brazilian                0.815993
admission_type_emergency  admission_loc_phys referral/normal deli    0.811482
admission_type_elective   admission_loc_phys referral/normal deli    0.810679
language_cape             ethnicity_black/cape verdean               0.706671
dtype: float64


In [54]:
# Removing highly correlated features
main = main.drop(['language_hait', 'discharge_loc_dead/expired'], axis = 1)

In [55]:
print(get_top_abs_correlations(main, 10))

avg_drg_mortality         avg_drg_severity                           0.955766
religion_buddhist         language_camb                              0.865491
admission_type_elective   admission_type_emergency                   0.851231
language_engl             language_nan                               0.824003
language_port             ethnicity_white - brazilian                0.815993
admission_type_emergency  admission_loc_phys referral/normal deli    0.811482
admission_type_elective   admission_loc_phys referral/normal deli    0.810679
language_cape             ethnicity_black/cape verdean               0.706671
religion_muslim           language_urdu                              0.705796
insurance_medicare        insurance_private                          0.697338
dtype: float64


In [56]:
y = main['depression']
y.shape

(813,)

In [57]:
X = main.drop(['depression'], axis=1)
X.shape

(813, 98)

## Splitting the dataset into the Training set and Test set

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling - Standardization vs. Normalization

Feature scaling is essential for machine learning algorithms that calculate distances between data. Therefore, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance.

* Normalization is recommended when you have a normally distributed observations.
* Standardization works all the time. (recommended)
* We need to perform Feature Scaling when we are dealing with Gradient Descent Based algorithms (Linear and Logistic Regression, Neural Network) and Distance-based algorithms (KNN, K-means, SVM) as these are very sensitive to the range of the data points.

* It is a good practice to fit the scaler on the training data and then use it to transform the testing data. This would avoid any data leakage during the model testing process. Also, the scaling of target values is generally not required.
* Only apply standardization to numerical columns and not the other One-Hot Encoded features. Standardizing the One-Hot encoded features would mean assigning a distribution to categorical features. You don’t want to do that! While it is fine to apply normalization to all kinds of columns including One-Hot Encorded features because One-Hot encoded features are already in the range between 0 to 1. So, normalization would not affect their value.

In [99]:
# Normalization 
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transofrm teting data
X_test_norm = norm.transform(X_test)

In [100]:
# Standardization with sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()

# numerical features
num_cols = ['length_admit', 'avg_drg_mortality', 'avg_drg_severity']

# apply standardization on numerical features
for i in num_cols:
    scale = StandardScaler().fit(X_train_stand[[i]])
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

## Dimensionality Reduction

Dimensionality reduction is a data preparation technique performed on data prior to modeling. It might be performed after data cleaning and data scaling and before training a predictive model.

As such, any dimensionality reduction performed on training data must also be performed on new data, such as a test dataset, validation dataset, and data when making a prediction with the final model.

### Principal Component Analysis (PCA)

Standardization: All the variables should be on the same scale before applying PCA, otherwise, a feature with large values will dominate the result.

In [101]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
X_train_norm_pca = pca.fit_transform(X_train_norm)
X_test_norm_pca = pca.transform(X_test_norm)
X_train_stand_pca = pca.fit_transform(X_train_stand)
X_test_stand_pca = pca.transform(X_test_stand)

##### Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_norm_pca, y_train)

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test_norm_pca)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[132   3]
 [ 26   2]]


0.8220858895705522

In [63]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_stand_pca, y_train)

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test_stand_pca)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[133   2]
 [ 27   1]]


0.8220858895705522

##### KNN

In [64]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # for Euclidean distance
classifier.fit(X_train_norm_pca, y_train)

# predicting the test set results
y_pred = classifier.predict(X_test_norm_pca)

# making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[120  15]
 [ 18  10]]


0.7975460122699386

In [65]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # for Euclidean distance
classifier.fit(X_train_stand_pca, y_train)

# predicting the test set results
y_pred = classifier.predict(X_test_stand_pca)

# making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[124  11]
 [ 21   7]]


0.803680981595092

#### Support Vector Machine

Assumption: observations are linear

In [66]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_norm_pca, y_train)

y_pred = classifier.predict(X_test_norm_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[135   0]
 [ 28   0]]


0.8282208588957055

In [67]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_stand_pca, y_train)

y_pred = classifier.predict(X_test_stand_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[135   0]
 [ 28   0]]


0.8282208588957055

#### Kernel SVM

Maaping to a higher dimensional space can be highly compute-intensive.

Types of Kernal Functions:
* Gaussian RBF Kernel
* Sigmoid Kernel
* Polynomial Kernel
* and etc.

In [68]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_norm_pca, y_train)

y_pred = classifier.predict(X_test_norm_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[135   0]
 [ 28   0]]


0.8282208588957055

In [69]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_stand_pca, y_train)

y_pred = classifier.predict(X_test_stand_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[135   0]
 [ 28   0]]


0.8282208588957055

#### Naive Bayes

Advantages: This algorithm requires a small amount of training data to estimate the necessary parameters. Naive Bayes classifiers are extremely fast compared to more sophisticated methods.

Disadvantages: Naive Bayes is is known to be a bad estimator.

In [70]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train_norm_pca, y_train)

y_pred = classifier.predict(X_test_norm_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[130   5]
 [ 26   2]]


0.8098159509202454

In [71]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train_stand_pca, y_train)

y_pred = classifier.predict(X_test_stand_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[131   4]
 [ 26   2]]


0.8159509202453987

#### Decision Tree

Advantages: Decision Tree is simple to understand and visualise, requires little data preparation, and can handle both numerical and categorical data.

Disadvantages: Decision tree can create complex trees that do not generalise well, and decision trees can be unstable because small variations in the data might result in a completely different tree being generated.

In [72]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train_norm_pca, y_train)

y_pred = classifier.predict(X_test_norm_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[108  27]
 [ 18  10]]


0.7239263803680982

In [73]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train_stand_pca, y_train)

y_pred = classifier.predict(X_test_stand_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[95 40]
 [14 14]]


0.6687116564417178

#### Random Forest

Ensemble Learning : using different machine algorithms 

The algorithm does not work well for datasets having a lot of outliers, something which needs addressing prior to the model building.

Advantages: Reduction in over-fitting and random forest classifier is more accurate than decision trees in most cases.

Disadvantages: Slow real time prediction, difficult to implement, and complex algorithm.

Build on top of Decision Trees

In [74]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train_norm_pca, y_train)

y_pred = classifier.predict(X_test_norm_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[126   9]
 [ 23   5]]


0.803680981595092

In [75]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train_stand_pca, y_train)

y_pred = classifier.predict(X_test_stand_pca)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[127   8]
 [ 25   3]]


0.7975460122699386

## Feature Scaling II

In [76]:
# Normalization 
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transofrm teting data
X_test_norm = norm.transform(X_test)

In [77]:
# Standardization with sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()

# numerical features
num_cols = ['length_admit', 'avg_drg_mortality', 'avg_drg_severity']

# apply standardization on numerical features
for i in num_cols:
    scale = StandardScaler().fit(X_train_stand[[i]])
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

### Linear Discriminant Analysis (LDA)

In [78]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 1)
X_train_norm_lda = lda.fit_transform(X_train_norm, y_train)
X_test_norm_lda = lda.transform(X_test_norm)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 1)
X_train_stand_lda = lda.fit_transform(X_train_stand, y_train)
X_test_stand_lda = lda.transform(X_test_stand)

#### Logistic Regression

In [79]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_norm_lda, y_train)

# predicting the test set results
y_pred = classifier.predict(X_test_norm_lda)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[119  16]
 [ 23   5]]


0.7607361963190185

In [80]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_stand_lda, y_train)

# predicting the test set results
y_pred = classifier.predict(X_test_stand_lda)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[119  16]
 [ 23   5]]


0.7607361963190185

#### K Nearest Neighbor (KNN)

K-Nearest Neighbor (KNN) algorithm predicts based on the specified number (k) of the nearest neighboring data points. Here, the pre-processing of the data is significant as it impacts the distance measurements directly. Unlike others, the model does not have a mathematical formula, neither any descriptive ability.

It is a simple, fairly accurate model preferable mostly for smaller datasets, owing to huge computations involved on the continuous predictors.

Step 1: Choose the number of K of neighbors

Step 2: Take the K nearest neighbors of the new data point, according to the Euclidean distance

Step 3: Among these K neighbors, count the number of data points in each category

Step 4: Assign the new data point ot the category where you counted the most neighbors

Then, the model will be ready. 

In [81]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # for Euclidean distance
classifier.fit(X_train_norm_lda, y_train)

# predicting the test set results
y_pred = classifier.predict(X_test_norm_lda)

# making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[119  16]
 [ 22   6]]


0.7668711656441718

In [82]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2) # for Euclidean distance
classifier.fit(X_train_stand_lda, y_train)

# predicting the test set results
y_pred = classifier.predict(X_test_stand_lda)

# making the confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[119  16]
 [ 22   6]]


0.7668711656441718

#### Support Vector Machine

Assumption: observations are linear

In [83]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_norm_lda, y_train)

y_pred = classifier.predict(X_test_norm_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[127   8]
 [ 25   3]]


0.7975460122699386

In [84]:
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_stand_lda, y_train)

y_pred = classifier.predict(X_test_stand_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[127   8]
 [ 25   3]]


0.7975460122699386

#### Kernel SVM

Maaping to a higher dimensional space can be highly compute-intensive.

Types of Kernal Functions:
* Gaussian RBF Kernel
* Sigmoid Kernel
* Polynomial Kernel
* and etc.

In [85]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_norm_lda, y_train)

y_pred = classifier.predict(X_test_norm_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[127   8]
 [ 25   3]]


0.7975460122699386

In [86]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train_stand_lda, y_train)

y_pred = classifier.predict(X_test_stand_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[127   8]
 [ 25   3]]


0.7975460122699386

#### Naive Bayes

Advantages: This algorithm requires a small amount of training data to estimate the necessary parameters. Naive Bayes classifiers are extremely fast compared to more sophisticated methods.

Disadvantages: Naive Bayes is is known to be a bad estimator.

In [87]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train_norm_lda, y_train)

y_pred = classifier.predict(X_test_norm_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[119  16]
 [ 23   5]]


0.7607361963190185

In [88]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train_stand_lda, y_train)

y_pred = classifier.predict(X_test_stand_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[119  16]
 [ 23   5]]


0.7607361963190185

#### Decision Tree

Advantages: Decision Tree is simple to understand and visualise, requires little data preparation, and can handle both numerical and categorical data.

Disadvantages: Decision tree can create complex trees that do not generalise well, and decision trees can be unstable because small variations in the data might result in a completely different tree being generated.

In [89]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train_norm_lda, y_train)

y_pred = classifier.predict(X_test_norm_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[93 42]
 [17 11]]


0.6380368098159509

In [90]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train_stand_lda, y_train)

y_pred = classifier.predict(X_test_stand_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[93 42]
 [17 11]]


0.6380368098159509

#### Random Forest

Ensemble Learning : using different machine algorithms 

The algorithm does not work well for datasets having a lot of outliers, something which needs addressing prior to the model building.

Advantages: Reduction in over-fitting and random forest classifier is more accurate than decision trees in most cases.

Disadvantages: Slow real time prediction, difficult to implement, and complex algorithm.

Build on top of Decision Trees

In [91]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train_norm_lda, y_train)

y_pred = classifier.predict(X_test_norm_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[97 38]
 [19  9]]


0.6503067484662577

In [92]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train_stand_lda, y_train)

y_pred = classifier.predict(X_test_stand_lda)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[97 38]
 [19  9]]


0.6503067484662577

## Notes:

1. Try GridSearch with cross validation